### Customer Lifetime Value (CLV) - Base & On-off case

In [6]:
def clv_base(T, i, m):
    """Base CLV model: Defined time horizon"""
    result = 0
    for t in range(T):
        current_m = m[t]
        current_discount = 1 / (1 + i)**(t + 1) #raised to t + 1 to circumvent the 0 indexing
        result += current_m * current_discount
    return result

def clv_on_off(m, i, r):
    """On-off CLV model: Infinite time horizon"""
    discount = r / (1 + i - r)
    return m * discount

def p_act(t, T_acq, T_p, n):
    """Calculates the purchase probability of a customer"""
    top = T_p - T_acq
    bottom = t - T_acq
    return (top / bottom)**n

def clv_rec_freq(i, T, m, T_acq, T_p, n):
    """Calculates the CLV with recency and frequency effect"""
    result = 0
    for t in range(T):
        discount = m / (1 + i)**(t + 1) #raised to t + 1 to circumvent the 0 indexing
        result += p_act(t + 1, T_acq, T_p, n) * discount
    return result

In [7]:
T = 4
m = [6, 3, 4, 2]
i = 0.04

print(clv_base(T, i, m))
print(clv_on_off(4, i, 0.87))
print(p_act(5, -12, -3, 4))
print(clv_rec_freq(0.05, 3, 30, -12, -3, 4))

13.808493225027133
20.470588235294112
0.07855509392847308
14.56929454207649


### Markov-chain

In [8]:
import numpy as np
from numpy.linalg import matrix_power

In [9]:
def markov_chain(P, R, periods):
    """Markov chain calcultor"""
    result = np.zeros((3, 1))
    for period in range(periods):
        if period == 0:
            current_P = np.identity(len(P))
        else:
            current_P = matrix_power(P, period)
        current_term = np.matmul(current_P, R)
        result = np.add(result, current_term)
    return result

In [10]:
P = np.array( #transition matrix
    [[0.999, 0.001, 0],
    [0, 0.8, 0.2],
    [0, 0, 1]])

R = np.array( #revenue matrix
    [[0],
    [30],
    [0]])

P_1 = np.array( #transition matrix
    [[0.99, 0.01, 0],
    [0, 0.8, 0.2],
    [0, 0, 1]])

R_1 = np.array( #revenue matrix
    [[-0.2],
    [30],
    [0]])

print(markov_chain(P, R, 400))
print(markov_chain(P_1, R_1, 400))

[[ 48.96694884]
 [150.        ]
 [  0.        ]]
[[127.52471318]
 [150.        ]
 [  0.        ]]
